# 🔄 nanochat Backward/Bidirectional Language Models

This notebook lets you run inference with nanochat models trained in different directions:

| Model | Direction | Description |
|-------|-----------|-------------|
| `nanochat-760M-backward` | Backward | Right-to-left prediction |
| `nanochat-760M-bidirectional` | Bidirectional | Both directions with special tokens |
| `nanochat-760M-backward-sft` | Backward | Chat fine-tuned backward model |

**Repository:** [github.com/traghav/onanchat](https://github.com/traghav/onanchat)

---

## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q torch huggingface_hub

In [ ]:
# Clone the onanchat repository for model code
!git clone https://github.com/traghav/onanchat.git
%cd onanchat

In [ ]:
import os
import sys
import torch
import pickle
from huggingface_hub import hf_hub_download

# Add to path
sys.path.insert(0, '.')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Select and Download Model

Choose which model to use:

In [ ]:
#@title Select Model
MODEL_CHOICE = "nanochat-760M-backward" #@param ["nanochat-760M-backward", "nanochat-760M-bidirectional", "nanochat-760M-backward-sft"]

REPO_ID = f"raghavt/{MODEL_CHOICE}"
print(f"Selected model: {REPO_ID}")

In [ ]:
# Download model files from HuggingFace
print("Downloading model files...")

model_path = hf_hub_download(repo_id=REPO_ID, filename="model.pt")
meta_path = hf_hub_download(repo_id=REPO_ID, filename="meta.json")
tokenizer_path = hf_hub_download(repo_id=REPO_ID, filename="tokenizer.pkl")
token_bytes_path = hf_hub_download(repo_id=REPO_ID, filename="token_bytes.pt")

print(f"✓ Model downloaded to: {model_path}")
print(f"✓ Metadata downloaded to: {meta_path}")
print(f"✓ Tokenizer downloaded to: {tokenizer_path}")

## 3. Load Model and Tokenizer

In [ ]:
import json
from nanochat.gpt import GPT, GPTConfig

# Load metadata
with open(meta_path, 'r') as f:
    meta = json.load(f)

direction = meta.get('direction', 'forward')
model_config = meta['model_config']

print(f"Model direction: {direction}")
print(f"Model config: {json.dumps(model_config, indent=2)}")

In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Create model
config = GPTConfig(
    sequence_len=model_config['sequence_len'],
    vocab_size=model_config['vocab_size'],
    n_layer=model_config['n_layer'],
    n_head=model_config['n_head'],
    n_kv_head=model_config['n_kv_head'],
    n_embd=model_config['n_embd']
)

model = GPT(config)

# Load weights
print("Loading model weights...")
state_dict = torch.load(model_path, map_location=device, weights_only=False)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"✓ Model loaded: {num_params/1e6:.1f}M parameters")

In [ ]:
# Load tokenizer
from nanochat.tokenizer import Tokenizer

# Copy tokenizer files to expected location
os.makedirs(os.path.expanduser('~/.cache/nanochat/tokenizer'), exist_ok=True)
import shutil
shutil.copy(tokenizer_path, os.path.expanduser('~/.cache/nanochat/tokenizer/tokenizer.pkl'))
shutil.copy(token_bytes_path, os.path.expanduser('~/.cache/nanochat/tokenizer/token_bytes.pt'))

tokenizer = Tokenizer()
print(f"✓ Tokenizer loaded with vocab size: {tokenizer.vocab_size}")

## 4. Inference Functions

In [ ]:
@torch.no_grad()
def generate(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 100,
    temperature: float = 0.8,
    top_k: int = 50,
    direction: str = "forward"
):
    """
    Generate text from a prompt.
    
    For backward models:
    - Input prompt is the "ending" of the text
    - Model generates what came "before"
    - Output is reversed for display
    """
    model.eval()
    
    # Encode prompt
    tokens = tokenizer.encode(prompt)
    
    # For backward models, reverse the input tokens
    if direction == "backward":
        tokens = tokens[::-1]
    
    # For bidirectional, add direction token
    if direction == "bidirectional":
        # Use forward direction by default for bidirectional
        forward_token = tokenizer.special_tokens.get('<|forward|>')
        if forward_token:
            tokens = [forward_token] + tokens
    
    # Add BOS token
    bos_token = tokenizer.special_tokens.get('<|begin_of_text|>', 0)
    tokens = [bos_token] + tokens
    
    # Convert to tensor
    x = torch.tensor([tokens], dtype=torch.long, device=device)
    
    # Generate
    generated_tokens = []
    for _ in range(max_new_tokens):
        # Get logits
        logits = model(x)[0, -1, :]  # Last position logits
        
        # Apply temperature
        logits = logits / temperature
        
        # Top-k sampling
        if top_k > 0:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[-1]] = float('-inf')
        
        # Sample
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Check for EOS
        eos_token = tokenizer.special_tokens.get('<|end_of_text|>')
        if eos_token and next_token.item() == eos_token:
            break
        
        generated_tokens.append(next_token.item())
        x = torch.cat([x, next_token.unsqueeze(0)], dim=1)
        
        # Truncate if too long
        if x.size(1) > config.sequence_len:
            x = x[:, -config.sequence_len:]
    
    # Decode generated tokens
    if direction == "backward":
        # Reverse the generated tokens for display
        generated_tokens = generated_tokens[::-1]
        generated_text = tokenizer.decode(generated_tokens)
        return generated_text + prompt  # Generated text comes before prompt
    else:
        generated_text = tokenizer.decode(generated_tokens)
        return prompt + generated_text

print("✓ Generation function defined")

## 5. Try It Out!

### Forward Generation (Standard)
Give a beginning, model generates what comes next.

### Backward Generation
Give an ending, model generates what came before!

In [ ]:
#@title Generate Text
prompt = "The quick brown fox" #@param {type:"string"}
max_tokens = 50 #@param {type:"slider", min:10, max:200, step:10}
temperature = 0.8 #@param {type:"slider", min:0.1, max:2.0, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:100, step:1}

print(f"Direction: {direction}")
print(f"Prompt: {prompt}")
print("-" * 50)

output = generate(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=max_tokens,
    temperature=temperature,
    top_k=top_k,
    direction=direction
)

print(f"\nGenerated text:\n{output}")

### Example: Backward Model Usage

With a backward model, you provide the *ending* and the model generates what *came before*:

In [ ]:
if direction == "backward":
    # Example: Give an ending, get the beginning
    ending = "and they lived happily ever after."
    
    print(f"Ending (your input): '{ending}'")
    print("-" * 50)
    
    story = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=ending,
        max_new_tokens=100,
        temperature=0.9,
        top_k=50,
        direction=direction
    )
    
    print(f"\nGenerated story (beginning + your ending):\n{story}")
else:
    print(f"Current model is {direction}, not backward.")
    print("Select 'nanochat-760M-backward' or 'nanochat-760M-backward-sft' to try backward generation.")

## 6. Interactive Chat (for SFT model)

In [ ]:
def chat(user_message: str, max_tokens: int = 150, temperature: float = 0.8):
    """
    Simple chat interface for the SFT model.
    """
    # Format as chat
    prompt = f"User: {user_message}\nAssistant:"
    
    output = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_k=50,
        direction=direction
    )
    
    # Extract assistant response
    if "Assistant:" in output:
        response = output.split("Assistant:")[-1].strip()
        # Stop at next "User:" if present
        if "User:" in response:
            response = response.split("User:")[0].strip()
        return response
    return output

if "sft" in MODEL_CHOICE:
    print("Chat interface ready! Use the chat() function.")
    print("Example: chat('What is the capital of France?')")
else:
    print(f"Note: Current model ({MODEL_CHOICE}) is a base model, not SFT.")
    print("For better chat, select 'nanochat-760M-backward-sft'")

In [ ]:
#@title Chat with the model
user_input = "Tell me a short story about a robot." #@param {type:"string"}

if "sft" in MODEL_CHOICE:
    response = chat(user_input)
    print(f"User: {user_input}")
    print(f"\nAssistant: {response}")
else:
    # For base models, just do completion
    output = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=user_input,
        max_new_tokens=100,
        temperature=0.8,
        top_k=50,
        direction=direction
    )
    print(f"Completion:\n{output}")

---

## About These Models

These models are part of a research project studying how LLMs learn when trained in different directions:

- **Backward models** predict tokens right-to-left instead of left-to-right
- **Bidirectional models** can switch between both directions using special tokens

### Research Questions:
1. Do backward models learn different internal representations?
2. Can models transfer knowledge across directions?
3. Does bidirectional training help both directions?

### Links:
- **Code:** [github.com/traghav/onanchat](https://github.com/traghav/onanchat)
- **Models:** [huggingface.co/raghavt](https://huggingface.co/raghavt)

Based on [nanochat](https://github.com/karpathy/nanochat) by Andrej Karpathy.